In [ ]:
# install several required packages, including Model Maker package
pip install -q tflite-model-maker

     |████████████████████████████████| 616 kB 5.2 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 41.8 MB/s 
     |████████████████████████████████| 6.4 MB 30.0 MB/s 
     |████████████████████████████████| 1.2 MB 39.9 MB/s 
     |████████████████████████████████| 840 kB 46.4 MB/s 
     |████████████████████████████████| 234 kB 25.1 MB/s 
     |████████████████████████████████| 3.4 MB 29.1 MB/s 
     |████████████████████████████████| 120 kB 48.7 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 1.1 MB 25.1 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 352 kB 50.4 MB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 
     |████████████████████████████████| 47.8 MB 66 kB/s 
     |████████████████████████████████| 462 kB 45.8 MB/s 
     |████████████████

In [ ]:
# Import the required packages

import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [ ]:
import pathlib
image_path = "/content/drive/MyDrive/GrapeDoc_Dataset/train"
image_path = pathlib.Path(image_path)

In [ ]:
# Use DataLoader class to load data
# As for from_folder() method, it could load data from the folder
# JPEG-encoded images and PNG-encoded images are supported.
data = DataLoader.from_folder(image_path)

INFO:tensorflow:Load image with size: 7252, num_label: 4, labels: Grape___Black_rot, Grape___Esca_(Black_Measles), Grape___Leaf_blight_(Isariopsis_Leaf_Spot), Grape___healthy.


In [ ]:
# Split it to training data (80%), validation data (10%, optional) and testing data (10%).
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
# Show 25 image examples with labels.
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Connect google drive to google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a custom image classifier model based on the loaded data
model = image_classifier.create(train_data, validation_data=validation_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 3,418,148
Trainable params: 5,124
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


181/181 [==============================] - 594s 3s/step - loss: 0.5477 - accuracy: 0.9173 - val_loss: 0.4499 - val_accuracy: 0.9793
Epoch 2/5
181/181 [==============================] - 40s 222ms/step - loss: 0.4450 - accuracy: 0.9767 - val_loss: 0.4245 - val_accuracy: 0.9834
Epoch 3/5
181/181 [==============================] - 40s 222ms/step - loss: 0.4331 - accuracy: 0.9812 - val_loss: 0.4174 - val_accuracy: 0.9890
Epoch 4/5
181/181 [==============================] - 41s 225ms/step - loss: 0.4282 - accuracy: 0.9855 - val_loss: 0.4132 - val_accuracy: 0.9903
Epoch 5/5
181/181 [==============================] - 40s 221ms/step - loss: 0.4234 - accuracy: 0.9848 - val_loss: 0.4102 - val_accuracy: 0.9876


In [ ]:
# Detailed model structure
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 3,418,148
Trainable params: 5,124
Non-trainable params: 3,413,024
_________________________________________________________________


In [ ]:
# Evaluate the result of the model, get the loss and accuracy of the model.
loss, accuracy = model.evaluate(test_data)

23/23 [==============================] - 76s 3s/step - loss: 0.4052 - accuracy: 0.9931


In [ ]:
# We could plot the predicted results in 100 test images. Predicted labels with red color are the wrong predicted results while others are correct.

# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(50, 50))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Convert the trained model to TensorFlow Lite model format with metadata
model.export(export_dir='/content/drive/MyDrive/Test11')

INFO:tensorflow:Assets written to: /tmp/tmpqwtikvfj/assets


INFO:tensorflow:Assets written to: /tmp/tmpqwtikvfj/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpqegmklcp/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpqegmklcp/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/Test11/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/Test11/model.tflite


In [ ]:
# exporting only the label file.
model.export(export_dir='/content/drive/MyDrive/Test11', export_format=ExportFormat.LABEL)

INFO:tensorflow:Saving labels in /content/drive/MyDrive/Test11/labels.txt


INFO:tensorflow:Saving labels in /content/drive/MyDrive/Test11/labels.txt


In [ ]:
# evaluate the tflite model with the evaluate_tflite method.
model.evaluate_tflite('/content/drive/MyDrive/Test11/model.tflite', test_data)

{'accuracy': 0.9931129476584022}